# Parallelisierung am Beispiel des Ising-Modells

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Dieses Jupyter-Notebook basiert auf dem Jupyter-Notebook [4-04-Ising-Modell.ipynb](4-04-Ising-Modell.ipynb) zum Ising-Modell und demonstriert die Parallelisierung eines Python-Programms. Im Folgenden werden nur die neuen Codeteile kommentiert. Für die Beschreibung des Codes zum Ising-Modell sei auf das Jupyter-Notebook [4-04-Ising-Modell.ipynb](4-04-Ising-Modll.ipynb) verwiesen.

## Importanweisungen

Die für die Parallelisierung wesentliche Importanweisung betrifft das Modul `concurrent.futures` der Python-Standardbilbiothek. Außerdem werden wir das `os`-Modul der Python-Standardbibliothek verwenden, um die Anzahl der logischen CPUs zu bestimmen.

In [ ]:
import concurrent.futures
import os
import time
from math import exp
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

plt.style.use("numphyspy.style")

## Temperaturabhängigkeit von spezifischer Wärme und magnetischer Suszeptibilität des Ising-Modells

### Bestimmung der nächsten Nachbarn

In [ ]:
def neighbours(size, dimension):
    neighbour = np.empty((size**dimension, 2*dimension),
                         dtype=int)
    index_shape = (size,)*dimension
    index = np.arange(size**dimension).reshape(index_shape)
    column = 0
    for axis in range(dimension):
        for shift in (-1, 1):
            neighbour[:, column] = np.roll(index, shift,
                                           axis=axis
                                           ).flatten()
            column = column + 1
    return neighbour

### Erzeugung des Anfangszustands

In [ ]:
rng = np.random.default_rng(123456)

def initial_state(n_spins, neighbour):
    state = rng.choice((-1, 1), size=n_spins)
    magnetization = np.sum(state)
    sum_nbr_spins = np.sum(state[neighbour], axis=1)
    energy = -np.sum(state*sum_nbr_spins)/2
    return state, energy, magnetization

### Iterationsschritt im Wolff-Algorithmus

In [ ]:
def wolff_generator(size, dimension, beta):
    n_spins = size**dimension
    neighbour = neighbours(size, dimension)
    p_limit = 1-exp(-2*beta)
    state, energy, magnetization = initial_state(n_spins,
                                                 neighbour)
    while True:
        yield energy, magnetization
        n_start = rng.choice(n_spins)
        to_do = {n_start}
        cluster = {n_start}
        state[n_start] = -state[n_start]
        while to_do:
            n_spin = to_do.pop()
            for nbr in neighbour[n_spin, :]:
                if (state[nbr] != state[n_spin]
                        and np.random.uniform() < p_limit):
                    to_do.add(nbr)
                    cluster.add(nbr)
                    state[nbr] = -state[nbr]
        delta_m = 2*state[n_start]*len(cluster)
        magnetization = magnetization + delta_m
        delta_e = 0
        for n_spin in cluster:
            sum_nbr_spins = 0
            for n_nbr_spin in neighbour[n_spin, :]:
                if n_nbr_spin not in cluster:
                    sum_nbr_spins += state[n_nbr_spin]
            delta_e = delta_e - 2*state[n_spin]*sum_nbr_spins
        energy = energy + delta_e

### Energie, Magnetisierung sowie deren erste und zweite Momente

In [ ]:
def thermo_values(args):
    (size, dimension, n_steps_init,
        n_steps, absmagn, beta) = args
    n_spins = size**dimension
    state = wolff_generator(size, dimension, beta)
    for n in range(n_steps_init):
        next(state)
    e_sum = 0
    e2_sum = 0
    m_sum = 0
    m2_sum = 0
    for n, (energy, magnetization) in zip(range(n_steps),
                                          state):
        e_sum = e_sum + energy
        e2_sum = e2_sum + energy**2
        if absmagn:
            magnetization = abs(magnetization)
        m_sum = m_sum + magnetization
        m2_sum = m2_sum + magnetization**2
    n_spins = size**dimension
    spec_heat = e2_sum/n_steps-(e_sum/n_steps)**2
    spec_heat = spec_heat * beta**2 / n_spins
    magn_susc = m2_sum/n_steps-(m_sum/n_steps)**2
    magn_susc = magn_susc * beta / n_spins
    return 1/beta, spec_heat, magn_susc

### Parallelisierte Berechnung für verschiedene Temperaturen

Um die parallelisierten Berechnungen vorzubereiten, wird zunächst eine Liste `parameters` von Parametersätzen erstellt. Zur Organisation der Python-Prozesse wird ein `ProcessPoolExecutor` in einem `with`-Kontext verwendet, in dem die in `parameters` definierten Aufgaben dann mit Hilfe von `map` auf die Python-Prozesse verteilt werden. Die Ergebnisse werden anschließend in den Listen `temperature_values`, `spec_heat_values` und `magn_susc_values` zusammengeführt. Zusätzlich werden die Ergebnisse auf dem Bildschirm ausgegeben.

In [ ]:
def thermo_values_of_T(size, dimension, n_steps_init,
                       n_steps, temperature_range, n_T,
                       absmagn, max_workers):
    beta_values = 1 / np.linspace(*temperature_range, n_T)
    temperature_values = []
    spec_heat_values = []
    magn_susc_values = []
    if absmagn:
        print("  T      C/N      χ'/N")
    else:
        print("  T      C/N      χ/N")
    parameters = [(size, dimension, n_steps_init, n_steps,
                   absmagn, beta) for beta in beta_values]
    with concurrent.futures.ProcessPoolExecutor(
            max_workers=max_workers) as e:
        for temperature, spec_heat, magn_susc in e.map(
                thermo_values, parameters):
            print(f"{temperature:6.4f} {spec_heat:7.4f} "
                  f"{magn_susc:8.2f}")
            temperature_values.append(temperature)
            spec_heat_values.append(spec_heat)
            magn_susc_values.append(magn_susc)
    return (temperature_values, spec_heat_values,
            magn_susc_values)

### Implementierung der Bedienelemente und graphische Darstellung der spezifischen Wärme und der magnetischen Suszeptibilität als Funktion der Temperatur

Neben der Einstellung von Parametern im Zusammenhang mit dem Ising-Modell gibt es hier noch zusätzlich die Möglichkeit, mit einem Schieberegler die maximale Anzahl `max_workers` der Prozesse festzulegen.

Zusätzlich zur graphischen Ausgabe der temperaturabhängigen spezifischen Wärme und der magnetischen Suszeptibilität wird auch noch die Rechendauer angezeigt, um diese als Funktion der Anzahl der verwendeten Prozesse untersuchen zu können.

In [ ]:
wide_label = {"description_width": "initial"}
widget_dict = {"dimension":
               widgets.RadioButtons(
                   options=[("eindimensional", 1),
                            ("zweidimensional", 2)],
                   value=1, description="Dimension"),
               "size":
               widgets.IntSlider(
                   value=100, min=100, max=1000,
                   step=100,
                   description=r"$n_\text{spins}$"),
               "temperature_range":
               widgets.FloatRangeSlider(
                   value=(0.2, 5), min=0.05, max=5,
                   step=0.05,
                   description="Temperaturbereich",
                   style=wide_label),
               "n_T":
               widgets.IntSlider(
                   value=30, min=10, max=200, step=10,
                   description="Temperaturwerte",
                   style=wide_label),
               "log_n_steps_init":
               widgets.IntSlider(
                   value=3, min=1, max=7, step=1,
                   description=r"$\log_{10}"
                               r"(n_\text{init})$"),
               "log_n_steps":
               widgets.IntSlider(
                   value=4, min=3, max=9, step=1,
                   description=r"$\log_{10}"
                               r"(n_\text{steps})$"),
               "absmagn":
               widgets.RadioButtons(
                   options=[("M", False),
                            ("|M|", True)],
                   value=False,
                   description="Magnetisierung"),
               "max_workers":
               widgets.IntSlider(
                   value=os.cpu_count()//2,
                   min=1, max=os.cpu_count(), step=1,
                   description=r"$n_\text{CPU}$")
               }

def handle_dim_change(change):
    attributes = {1: {"max": 1000, "min": 50, "step": 50,
                      "value": 100},
                  2: {"step": 5, "min": 5, "max": 40,
                      "value": 20}}
    for k, v in attributes[change.new].items():
        setattr(widget_dict["size"], k, v)

widget_dict["dimension"].observe(handle_dim_change,
                                 names="value")

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_result(dimension, size, temperature_range, n_T,
                log_n_steps_init, log_n_steps, absmagn,
                max_workers):
    n_spins = size**dimension
    n_steps_init = 10**log_n_steps_init
    n_steps = 10**log_n_steps
    start_time = time.time()
    temperature_vals, spec_heat_vals, magn_susc_vals = (
        thermo_values_of_T(size, dimension, n_steps_init,
                           n_steps, temperature_range, n_T,
                           absmagn, max_workers))
    end_time = time.time()
    print(f"Rechenzeit: {end_time-start_time:6.1f} s")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.plot(temperature_vals, spec_heat_vals)
    ax1.set_xlabel("$T$")
    ax1.set_ylabel("$C/N$")
    if absmagn:
        ax2.plot(temperature_vals, magn_susc_vals)
        label_chi = r"$\chi'/N$"
    else:
        ax2.semilogy(temperature_vals, magn_susc_vals)
        ax2.semilogy(temperature_vals,
                     n_spins/np.array(temperature_vals))
        label_chi = r"$\chi/N$"
    ax2.set_xlabel("$T$")
    ax2.set_ylabel(label_chi)